In [1]:
import json
import pandas as pd
import numpy as np

from khipu.extended import *


from jms.io import read_table_to_peaks
from jms.dbStructures import ExperimentalEcpdDatabase, knownCompoundDatabase
from intervaltree import IntervalTree
from khipu.epdsConstructor import epdsConstructor
from khipu.extended import (
    isotope_search_patterns,
    extended_adducts,
    adduct_search_patterns,
    adduct_search_patterns_neg,
)

def build_khipu(ft_path, ion_mode, limit_iso=False):
    peaklist = read_table_to_peaks(
        ft_path,
        has_header=True,
        mz_col=1,
        rtime_col=2,
        feature_id=0,
        full_extract=True,
    )
    for p in peaklist:
        p["id"] = p["id_number"]
        p["representative_intensity"] = None

    if ion_mode == "pos":
        adducts = adduct_search_patterns
    elif ion_mode == "neg":
        adducts = adduct_search_patterns_neg

    ECCON = epdsConstructor(peaklist, ion_mode)
    if limit_iso:
        dict_empcpds = ECCON.peaks_to_epdDict(
            isotope_search_patterns[:3],
            adducts,
            extended_adducts,
            mz_tolerance_ppm=10,
            rt_tolerance=3,
            charges=[1,2,3],
        )
        return dict_empcpds
    else:
        dict_empcpds = ECCON.peaks_to_epdDict(
            isotope_search_patterns,
            adducts,
            extended_adducts,
            mz_tolerance_ppm=10,
            rt_tolerance=3,
            charges=[1,2,3],
        )
        return dict_empcpds

def detect_labelling(khipu_list, unlabeled_samples, labeled_samples, result_name, labeling_threshold=10, skip_isos=None):
    skip_isos = {"M0"} if skip_isos is None else skip_isos
    for khipu in khipu_list:
        detect_labelling_khipu(khipu, unlabeled_samples, labeled_samples, result_name, labeling_threshold, skip_isos)
    return khipu_list

def detect_labelling_khipu(khipu, unlabeled_samples, labeled_samples, result_name, labeling_threshold=10, skip_isos=None):
    skip_isos = {"M0"} if skip_isos is None else skip_isos
    good_isotopologues = 0
    bad_isotopologues = 0
    for peak in khipu["MS1_pseudo_Spectra"]:
        isotope = peak["isotope"]
        if isotope not in skip_isos:
            unlabeled_avg = np.mean([peak[us] for us in unlabeled_samples])
            labeled_avg = np.mean([peak[s] for s in labeled_samples])
            if labeled_avg > unlabeled_avg * labeling_threshold:
                good_isotopologues += 1
            else:
                bad_isotopologues += 1
    if "labeling_scores" not in khipu:
        khipu["labeling_scores"] = {}
    khipu["labeling_scores"][result_name] = {
        "good_labelling": good_isotopologues,
        "bad_labelling": bad_isotopologues
    }

def apply_heuristic(khipu_list):
    for khipu in khipu_list:
        for r, score_dict in khipu["labeling_scores"].items():
            confidence_level = 0
            if score_dict["good_labelling"] > 0:
                confidence_level = max(confidence_level, 1)
            if score_dict["good_labelling"] > score_dict["bad_labelling"] and score_dict["good_labelling"] > 0:
                confidence_level = max(confidence_level, 2)
            if score_dict["good_labelling"] > score_dict["bad_labelling"] and score_dict["good_labelling"] > 0 and score_dict["bad_labelling"] == 0:
                confidence_level = max(confidence_level, 3)
            score_dict["confidence_level"] = confidence_level

def measure_overlap_with_GEM(khipu_list, GEM="/Users/mitchjo/khipu/khipu/metabolicModel_az_HumanGEM_20220302_noCompartmentalization.json", mz_tol=5):
    GEM_data = json.load(open(GEM))
    GEM_mz_tree = IntervalTree()
    id_to_name = {}
    for cpd in GEM_data["list_of_compounds"]:
        id_to_name[cpd['id']] = cpd['name']
        mass = cpd["neutral_mono_mass"]
        if mass:
            mass_err = mass / 1e6 * mz_tol
            GEM_mz_tree.addi(mass - mass_err, mass + mass_err, cpd['id'])
    for khipu in khipu_list:
        if "in_GEM" not in khipu:
            khipu["in_GEM"] = False
        for r in GEM_mz_tree.at(khipu["neutral_formula_mass"]):
            if "GEM_annotations" not in khipu:
                khipu["GEM_annotations"] = []
            khipu["in_GEM"] = True
            khipu["GEM_annotations"].append(id_to_name[r.data])
    return khipu_list

def organize_samples(table):
    metadata = pd.read_excel("./Randomization_293T_isotopelabeling.xlsx")
    sample_dict = {}
    for sample in metadata.to_dict(orient='records'):
        t = None
        if "_1h_" in sample["Name"]:
            t = 1
        elif "_6h_" in sample["Name"]:
            t = 6
        elif "_12h_" in sample["Name"]:
            t = 12
        else:
            t = 0
        if t not in sample_dict:
            sample_dict[t] = []
        for c in table.columns:
            if sample["SampleID"] + "_" in c:
                sample_dict[t].append(c)
    return sample_dict

In [2]:
#293 T

data = pd.read_excel("/Users/mitchjo/Documents/Khipu_Analysis/Randomization_293T_isotopelabeling.xlsx")

hilic_neg_tablepath = "/Users/mitchjo/KhipuPaper/AsariResults/AsariResults_293T_hilic_neg_422195052/export/full_Feature_table.tsv"
hilic_neg_table = pd.read_csv(hilic_neg_tablepath, sep="\t")
hilic_neg_khipu = build_khipu(hilic_neg_tablepath, "neg")
hilic_neg_samples = organize_samples(hilic_neg_table)
detect_labelling(hilic_neg_khipu.values(), hilic_neg_samples[0], hilic_neg_samples[1], "1hr")
detect_labelling(hilic_neg_khipu.values(), hilic_neg_samples[0], hilic_neg_samples[6], "6hr")
detect_labelling(hilic_neg_khipu.values(), hilic_neg_samples[0], hilic_neg_samples[12], "12hr")
measure_overlap_with_GEM(hilic_neg_khipu.values())
apply_heuristic(hilic_neg_khipu.values())
json.dump(hilic_neg_khipu, open("293T_hilic_neg_khipu.json", 'w+'), indent=4)

hilic_pos_tablepath = "/Users/mitchjo/KhipuPaper/AsariResults/AsariResults_293T_hilic_pos_422195215/export/full_Feature_table.tsv"
hilic_pos_table = pd.read_csv(hilic_pos_tablepath, sep="\t")
hilic_pos_khipu = build_khipu(hilic_pos_tablepath, "pos")
hilic_pos_samples = organize_samples(hilic_pos_table)
detect_labelling(hilic_pos_khipu.values(), hilic_pos_samples[0], hilic_pos_samples[1], "1hr")
detect_labelling(hilic_pos_khipu.values(), hilic_pos_samples[0], hilic_pos_samples[6], "6hr")
detect_labelling(hilic_pos_khipu.values(), hilic_pos_samples[0], hilic_pos_samples[12], "12hr")
measure_overlap_with_GEM(hilic_pos_khipu.values())
apply_heuristic(hilic_pos_khipu.values())
json.dump(hilic_pos_khipu, open("293T_hilic_pos_khipu.json", 'w+'), indent=4)

rp_neg_tablepath = "/Users/mitchjo/KhipuPaper/AsariResults/AsariResults_293T_rp_neg_422195317/export/full_Feature_table.tsv"
rp_neg_table = pd.read_csv(rp_neg_tablepath, sep="\t")
rp_neg_khipu = build_khipu(rp_neg_tablepath, "neg")
rp_neg_samples = organize_samples(rp_neg_table)
detect_labelling(rp_neg_khipu.values(), rp_neg_samples[0], rp_neg_samples[1], "1hr")
detect_labelling(rp_neg_khipu.values(), rp_neg_samples[0], rp_neg_samples[6], "6hr")
detect_labelling(rp_neg_khipu.values(), rp_neg_samples[0], rp_neg_samples[12], "12hr")
measure_overlap_with_GEM(rp_neg_khipu.values())
apply_heuristic(rp_neg_khipu.values())
json.dump(rp_neg_khipu, open("293T_rp_neg_khipu.json", 'w+'), indent=4)

rp_pos_tablepath = "/Users/mitchjo/KhipuPaper/AsariResults/AsariResults_293T_rp_pos_422195448/export/full_Feature_table.tsv"
rp_pos_table = pd.read_csv(rp_pos_tablepath, sep="\t")
rp_pos_khipu = build_khipu(rp_pos_tablepath, "pos")
rp_pos_samples = organize_samples(rp_pos_table)
detect_labelling(rp_pos_khipu.values(), rp_pos_samples[0], rp_pos_samples[1], "1hr")
detect_labelling(rp_pos_khipu.values(), rp_pos_samples[0], rp_pos_samples[6], "6hr")
detect_labelling(rp_pos_khipu.values(), rp_pos_samples[0], rp_pos_samples[12], "12hr")
measure_overlap_with_GEM(rp_pos_khipu.values())
apply_heuristic(rp_pos_khipu.values())
json.dump(rp_pos_khipu, open("293T_rp_pos_khipu.json", 'w+'), indent=4)

st001776_path = "/Users/mitchjo/KhipuPaper/AsariResults/AsariResults_ST001776_hilic_pos_423103244/export/full_Feature_table.tsv"
st001776_table = pd.read_csv(st001776_path, sep="\t")
st001776_khipu = build_khipu(st001776_path, "pos")
measure_overlap_with_GEM(st001776_khipu.values())
json.dump(st001776_khipu, open("st001776.json", 'w+'), indent=4)

st001776_path = "/Users/mitchjo/KhipuPaper/AsariResults/AsariResults_ST001776_hilic_pos_423103244/export/full_Feature_table.tsv"
st001776_table = pd.read_csv(st001776_path, sep="\t")
st001776_khipu = build_khipu(st001776_path, "pos", limit_iso=True)
measure_overlap_with_GEM(st001776_khipu.values())
json.dump(st001776_khipu, open("st001776_limited.json", 'w+'), indent=4)



Multiple charges considered: [1, 2, 3]


Khipu search grid: 
                 M-H-       Na/H        HCl        K/H        ACN      HCOOH
M0          -1.007276  20.974724  34.969424  36.948606  40.019273  44.998204
13C/12C     -0.003921  21.978079  35.972779  37.951961  41.022628  46.001559
13C/12C*2    0.999434  22.981434  36.976134  38.955316  42.025983  47.004914
13C/12C*3    2.002789  23.984789  37.979489  39.958671  43.029338  48.008269
13C/12C*4    3.006144  24.988144  38.982844  40.962026  44.032693  49.011624
13C/12C*5    4.009499  25.991499  39.986199  41.965381  45.036048  50.014979
13C/12C*6    5.012854  26.994854  40.989554  42.968736  46.039403  51.018334
13C/12C*7    6.016209  27.998209  41.992909  43.972091  47.042758  52.021689
13C/12C*8    7.019564  29.001564  42.996264  44.975446  48.046113  53.025044
13C/12C*9    8.022919  30.004919  43.999619  45.978801  49.049468  54.028399
13C/12C*10   9.026274  31.008274  45.002974  46.982156  50.052823  55.031754
13C/12C*11  1

/Users/mitchjo/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/mitchjo/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Downsized input network with 145 features, highest peak at F9716 
Downsized input network with 186 features, highest peak at F9510 
Downsized input network with 98 features, highest peak at F9727 


/Users/mitchjo/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/mitchjo/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Downsized input network with 168 features, highest peak at F35911 


/Users/mitchjo/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/mitchjo/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/mitchjo/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/mitchjo/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Downsized input network with 151 features, highest peak at F36168 
Downsized input network with 122 features, highest peak at F32446 


/Users/mitchjo/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Constructed 3161 khipus in this round.




KeyboardInterrupt: 

In [28]:
def organize_AML(table):
    sample_dict = {}
    for sequence in ["./20240318-HILICpos-RPneg.csv", "./20240319-HILICneg-RPpos.csv"]:
        metadata = pd.read_csv(sequence)
        for sample in metadata.to_dict(orient='records'):
            t = None
            if "_2hr_" in sample["Sample ID"]:
                t = 2
            elif "_6hr_" in sample["Sample ID"]:
                t = 6
            elif "_12hr_" in sample["Sample ID"]:
                t = 12
            elif "12C_ctrl" in sample["Sample ID"]:
                t = 0
            if t is not None:
                if t not in sample_dict:
                    sample_dict[t] = []
                for c in table.columns:
                    if sample["File Name"] in c:
                        sample_dict[t].append(c)
    return sample_dict

In [39]:
#AML

data = pd.read_excel("/Users/mitchjo/Documents/Khipu_Analysis/Randomization_293T_isotopelabeling.xlsx")

hilic_neg_tablepath = "/Users/mitchjo/KhipuPaper/AsariResults/AsariResults_AML_hilic_neg_42595839/export/full_Feature_table.tsv"
hilic_neg_table = pd.read_csv(hilic_neg_tablepath, sep="\t")
hilic_neg_khipu = build_khipu(hilic_neg_tablepath, "neg")
hilic_neg_samples = organize_AML(hilic_neg_table)
detect_labelling(hilic_neg_khipu.values(), hilic_neg_samples[0], hilic_neg_samples[2], "2hr")
detect_labelling(hilic_neg_khipu.values(), hilic_neg_samples[0], hilic_neg_samples[6], "6hr")
detect_labelling(hilic_neg_khipu.values(), hilic_neg_samples[0], hilic_neg_samples[12], "12hr")
measure_overlap_with_GEM(hilic_neg_khipu.values())
apply_heuristic(hilic_neg_khipu.values())
json.dump(hilic_neg_khipu, open("AML_hilic_neg_khipu.json", 'w+'), indent=4)

hilic_pos_tablepath = "/Users/mitchjo/KhipuPaper/AsariResults/AsariResults_AML_hilic_pos_42595928/export/full_Feature_table.tsv"
hilic_pos_table = pd.read_csv(hilic_pos_tablepath, sep="\t")
hilic_pos_khipu = build_khipu(hilic_pos_tablepath, "pos")
hilic_pos_samples = organize_AML(hilic_pos_table)
detect_labelling(hilic_pos_khipu.values(), hilic_pos_samples[0], hilic_pos_samples[2], "2hr")
detect_labelling(hilic_pos_khipu.values(), hilic_pos_samples[0], hilic_pos_samples[6], "6hr")
detect_labelling(hilic_pos_khipu.values(), hilic_pos_samples[0], hilic_pos_samples[12], "12hr")
measure_overlap_with_GEM(hilic_pos_khipu.values())
apply_heuristic(hilic_pos_khipu.values())
json.dump(hilic_pos_khipu, open("AML_hilic_pos_khipu.json", 'w+'), indent=4)

rp_neg_tablepath = "/Users/mitchjo/KhipuPaper/AsariResults/AsariResults_AML_rp_neg_4251009/export/full_Feature_table.tsv"
rp_neg_table = pd.read_csv(rp_neg_tablepath, sep="\t")
rp_neg_khipu = build_khipu(rp_neg_tablepath, "neg")
rp_neg_samples = organize_AML(rp_neg_table)
detect_labelling(rp_neg_khipu.values(), rp_neg_samples[0], rp_neg_samples[2], "2hr")
detect_labelling(rp_neg_khipu.values(), rp_neg_samples[0], rp_neg_samples[6], "6hr")
detect_labelling(rp_neg_khipu.values(), rp_neg_samples[0], rp_neg_samples[12], "12hr")
measure_overlap_with_GEM(rp_neg_khipu.values())
apply_heuristic(rp_neg_khipu.values())
json.dump(rp_neg_khipu, open("AML_rp_neg_khipu.json", 'w+'), indent=4)

rp_pos_tablepath = "/Users/mitchjo/KhipuPaper/AsariResults/AsariResults_AML_rp_pos_42510123/export/full_Feature_table.tsv"
rp_pos_table = pd.read_csv(rp_pos_tablepath, sep="\t")
rp_pos_khipu = build_khipu(rp_pos_tablepath, "pos")
rp_pos_samples = organize_AML(rp_pos_table)
detect_labelling(rp_pos_khipu.values(), rp_pos_samples[0], rp_pos_samples[2], "2hr")
detect_labelling(rp_pos_khipu.values(), rp_pos_samples[0], rp_pos_samples[6], "6hr")
detect_labelling(rp_pos_khipu.values(), rp_pos_samples[0], rp_pos_samples[12], "12hr")
measure_overlap_with_GEM(rp_pos_khipu.values())
apply_heuristic(rp_pos_khipu.values())
json.dump(rp_pos_khipu, open("AML_rp_pos_khipu.json", 'w+'), indent=4)



Multiple charges considered: [1, 2, 3]


Khipu search grid: 
                 M-H-       Na/H        HCl        K/H        ACN      HCOOH
M0          -1.007276  20.974724  34.969424  36.948606  40.019273  44.998204
13C/12C     -0.003921  21.978079  35.972779  37.951961  41.022628  46.001559
13C/12C*2    0.999434  22.981434  36.976134  38.955316  42.025983  47.004914
13C/12C*3    2.002789  23.984789  37.979489  39.958671  43.029338  48.008269
13C/12C*4    3.006144  24.988144  38.982844  40.962026  44.032693  49.011624
13C/12C*5    4.009499  25.991499  39.986199  41.965381  45.036048  50.014979
13C/12C*6    5.012854  26.994854  40.989554  42.968736  46.039403  51.018334
13C/12C*7    6.016209  27.998209  41.992909  43.972091  47.042758  52.021689
13C/12C*8    7.019564  29.001564  42.996264  44.975446  48.046113  53.025044
13C/12C*9    8.022919  30.004919  43.999619  45.978801  49.049468  54.028399
13C/12C*10   9.026274  31.008274  45.002974  46.982156  50.052823  55.031754
13C/12C*11  1

/Users/mitchjo/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Downsized input network with 131 features, highest peak at F8165 
Constructed 1044 khipus in this round.


Khipu search grid: 
                        M-H-, 2x charged  Na/H, 2x charged  HCl, 2x charged  \
M0                             -0.503638          9.983724        16.981074   
13C/12C, 2x charged            -0.001961         10.485401        17.482751   
13C/12C*2, 2x charged           0.499717         10.987079        17.984429   
13C/12C*3, 2x charged           1.001394         11.488756        18.486106   
13C/12C*4, 2x charged           1.503072         11.990434        18.987784   
13C/12C*5, 2x charged           2.004749         12.492111        19.489461   
13C/12C*6, 2x charged           2.506427         12.993789        19.991139   
13C/12C*7, 2x charged           3.008104         13.495466        20.492816   
13C/12C*8, 2x charged           3.509782         13.997144        20.994494   
13C/12C*9, 2x charged           4.011459         14.498821        21.496171   
13C/

/Users/mitchjo/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Constructed 1668 khipus in this round.


Khipu search grid: 
                        M+H+, 2x charged  Na/H, 2x charged  HCl, 2x charged  \
M0                              0.503638         11.998276        18.995626   
13C/12C, 2x charged             1.005316         12.499954        19.497304   
13C/12C*2, 2x charged           1.506993         13.001631        19.998981   
13C/12C*3, 2x charged           2.008671         13.503309        20.500659   
13C/12C*4, 2x charged           2.510348         14.004986        21.002336   
13C/12C*5, 2x charged           3.012026         14.506664        21.504014   
13C/12C*6, 2x charged           3.513703         15.008341        22.005691   
13C/12C*7, 2x charged           4.015381         15.510019        22.507369   
13C/12C*8, 2x charged           4.517058         16.011696        23.009046   
13C/12C*9, 2x charged           5.018736         16.513374        23.510724   
13C/12C*10, 2x charged          5.520413         17.015051        24.0

/Users/mitchjo/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',




 ~~~~~~ Got 2537 khipus, with 8237 features ~~~~~~~ 


